# Simulator

In [1]:
import sys
sys.path.insert(0, "..")
import pandas as pd
import matplotlib.pyplot as plt
from trading_simulator import TradingSimulator, DLSimulator
from dl_solutions.dlsolutions import CryptoDLSolutions
from data_processor.data_processing import DataProcessor

Using TensorFlow backend.


## Classic

In [ ]:
cryptos = ['ADA', 'BTC', 'ETH', 'LTC', 'LNK']
sel = 0

In [ ]:
for crypto in cryptos:
    processor = DataProcessor([crypto])
    sim = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2, log = False)
    sim.simulate()

In [ ]:
processor = DataProcessor([crypto])
sim1 = TradingSimulator(processor, crypto, strategy = [1], loss_allowed = 0.2)
sim2 = TradingSimulator(processor, crypto, strategy = [2], loss_allowed = 0.2)
sim3 = TradingSimulator(processor, crypto, strategy = [3], loss_allowed = 0.2)
sim4 = TradingSimulator(processor, crypto, strategy = [4], loss_allowed = 0.2)
sim5 = TradingSimulator(processor, crypto, strategy = [5], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [6], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [7], loss_allowed = 0.2)
sim7 = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2)

In [ ]:
sim1.simulate()

In [ ]:
sim2.simulate()

In [ ]:
sim3.simulate()

In [ ]:
sim4.simulate()

In [ ]:
sim5.simulate()

In [ ]:
sim6.simulate()

In [ ]:
sim7.simulate()

## DL Solution

In [18]:
crypto = 'ETH'
periods_to_retraining = 1000
prev_periods = 5
pred_periods = 10
model_selector = 'lstm'
columns = ['RSI', 'close']
#columns = ['weekday_num', 'RSI', 'MACD', 'close']
#columns = ['RSI', 'MACD', 'close']
#columns = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'RSI', 'MACD', 'close']
#columns = ['SO_K', 'SMA 50', 'SMA 200', 'close']
num_features = len(columns)
target = None

processor = DataProcessor([crypto])
sim = DLSimulator(processor, crypto, periods_to_retraining, prev_periods, pred_periods, model_selector, columns, target)
# TODO Add target if we want features as timestamps
#sim.simulate()
df = sim.get_df()
real_df = df.loc[327:1577]
real_df = df.loc[327:1577]
to_pred_df = df.loc[1578:1578]
real_and_pred = df.iloc[:1001]
real_df

# PROBAR GENERAR DATASET CON SALTOS DE X DIAS A CONOCER

Loading... ETH
Extracting columns columns for ETH
Proccessing and arranging columns for LSTM model


,close_4,RSI_4,close_3,RSI_3,close_2,RSI_2,close_1,RSI_1,close_0,RSI_0,close,RSI
327,467.87,44.722439,470.47,45.380609,485.57,49.166169,486.77,49.465917,471.71,45.814704,468.57,49.010765
328,470.47,45.380609,485.57,49.166169,486.77,49.465917,471.71,45.814704,432.48,37.955422,449.04,44.652601
329,485.57,49.166169,486.77,49.465917,471.71,45.814704,432.48,37.955422,444.99,41.407223,460.23,47.531480
330,486.77,49.465917,471.71,45.814704,432.48,37.955422,444.99,41.407223,431.14,38.831477,457.47,46.883723
331,471.71,45.814704,432.48,37.955422,444.99,41.407223,431.14,38.831477,432.46,39.219538,450.05,45.104108
...,...,...,...,...,...,...,...,...,...,...,...,...
1573,4511.21,53.835840,4215.73,46.826530,4117.25,44.735958,4196.44,46.793026,4347.59,50.574650,3958.11,44.664534
1574,4215.73,46.826530,4117.25,44.735958,4196.44,46.793026,4347.59,50.574650,4306.40,49.541304,3875.42,42.891556
1575,4117.25,44.735958,4196.44,46.793026,4347.59,50.574650,4306.40,49.541304,4436.91,52.829940,3959.93,45.282169
1576,4196.44,46.793026,4347.59,50.574650,4306.40,49.541304,4436.91,52.829940,4105.64,44.841174,3924.14,44.433846


In [23]:
norm_strat = 2
strat = 1
layers = 4
neurons = [50, 50, 50, 50]
batch_size = 64
epochs = 10
if target != None: 
    num_timestamps = None
else:
    num_timestamps = prev_periods

lstm = CryptoDLSolutions(real_df, norm_strat, strat, layers, neurons, batch_size, epochs, num_timestamps, num_features)
lstm.build()
print(lstm.get_model().summary())
lstm.train()

Input shape: (1251, 5, 2) (1251, 2) (0, 5, 2) (0, 2)
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 5, 50)             10600     
_________________________________________________________________
lstm_26 (LSTM)               (None, 5, 50)             20200     
_________________________________________________________________
dropout_19 (Dropout)         (None, 5, 50)             0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 5, 50)             20200     
_________________________________________________________________
dropout_20 (Dropout)         (None, 5, 50)             0         
_________________________________________________________________
lstm_28 (LSTM)               (None, 50)                20200     
_________________________________________________________________
d

In [22]:
pred = lstm.predict()
print('pred:', pred)

testx (1, 5, 2)
preds [[0.85554487 0.5007516 ]]
preds rev norm [[4125.54216275   53.12055985]]
real [[4014.15         47.26817583]]
Test RMSE: 78.875
pred: [[4125.54216275   53.12055985]]


In [21]:
to_pred_df = df.loc[1578:1578]
to_pred_df

,close_4,RSI_4,close_3,RSI_3,close_2,RSI_2,close_1,RSI_1,close_0,RSI_0,close,RSI
1578,4306.4,49.541304,4436.91,52.82994,4105.64,44.841174,3897.94,40.686916,4089.37,45.681743,4014.15,47.268176


In [20]:
lstm.set_test(to_pred_df)

In [6]:
df[-20:]

,close_1,MACD_1,RSI_1,weekday_num_1,close_0,MACD_0,RSI_0,weekday_num_0,close,MACD,RSI,weekday_num
1569,4582.95,36.473333,55.716077,2.0,4511.21,43.536946,53.835840,3,4130.00,-57.803905,46.707531,6.0
1570,4511.21,43.536946,53.835840,3.0,4215.73,25.003914,46.826530,4,3781.93,-87.109146,39.777252,0.0
1571,4215.73,25.003914,46.826530,4.0,4117.25,2.342833,44.735958,5,3858.99,-102.929159,41.834910,1.0
1572,4117.25,2.342833,44.735958,5.0,4196.44,-9.121124,46.793026,6,4019.08,-101.380062,45.965427,2.0
1573,4196.44,-9.121124,46.793026,6.0,4347.59,-5.941362,50.574650,0,3958.11,-103.874758,44.664534,3.0
1574,4347.59,-5.941362,50.574650,0.0,4306.40,-6.668199,49.541304,1,3875.42,-111.241886,42.891556,4.0
1575,4306.40,-6.668199,49.541304,1.0,4436.91,3.249386,52.829940,2,3959.93,-109.004601,45.282169,5.0
1576,4436.91,3.249386,52.829940,2.0,4105.64,-15.443552,44.841174,3,3924.14,-108.864567,44.433846,6.0
1577,4105.64,-15.443552,44.841174,3.0,3897.94,-46.481683,40.686916,4,3944.73,-105.871725,45.071392,0.0
1578,3897.94,-46.481683,40.686916,4.0,4089.37,-54.998832,45.681743,5,4014.15,-96.782613,47.268176,1.0


In [ ]:
to_pred_df = pd.read_csv('/Users/gonzalozabala/Documents/Master IA/AIMaster/TFM/Code/crypto_bot/Datasets/ETH.csv', header=[1])
to_pred_df = pd.DataFrame(to_pred_df['close'].iloc[:10]).T
to_pred_df['y'] = 686
to_pred_df

In [ ]:
processor = DataProcessor(['ETH'])
processor.load_data()
processor.clean_data('ETH')
#processor.feature_extraction('ETH')
#columns = ['close', 'Open Close Difference', 'Volume USDT' ,'Result']
#columns = ['close']
#df = processor.feature_selection('ETH', columns) 
df.loc[1588]